### Importing libraries

In [1]:
from ImgOps import *
import copy as cp
from sklearn.cluster import DBSCAN

### Imreading images

In [2]:
query_img = cv.imread(r'query_4.jpg')
query_img_gray = cv.cvtColor(query_img, cv.COLOR_BGR2GRAY)
# imshow(query_img)

train_img = cv.imread(r'train_4.jpg')
train_img_gray = cv.cvtColor(train_img, cv.COLOR_BGR2GRAY)
# imshow(train_img)

### Adding keypoints via SIFT

In [3]:
# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
query_kps, query_des = sift.detectAndCompute(query_img_gray, None)
train_kps, train_des = sift.detectAndCompute(train_img_gray, None)

### Keypoints matching via FLANN

In [4]:
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(query_des, train_des, k=2)
matchesMask = [[0, 0] for _ in range(len(matches))]

good_matches = []
for i, (m, n) in enumerate(matches):
    if m.distance < .5 * n.distance:
        matchesMask[i] = [1, 0]
        good_matches.append(m)
good_matches = np.asarray(good_matches)

draw_params = dict(matchColor=(0, 255, 0),
                   matchesMask=matchesMask,
                   flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
matched_img = cv.drawMatchesKnn(query_img, query_kps, train_img, train_kps, matches, None, **draw_params)

# imshow(matched_img)

### Clustering via DBSCAN

In [5]:
# Dictionary(ptQuery_ptTrain) = { (x_q, y_q): (x_t), (y_t) }
ptQuery_ptTrain = {}
for DMatch in good_matches:
    pt_q = query_kps[DMatch.queryIdx].pt
    pt_t = train_kps[DMatch.trainIdx].pt
    ptQuery_ptTrain[pt_q] = pt_t


# Clustering
clusterized = DBSCAN(eps=50, min_samples=5).fit_predict(list(ptQuery_ptTrain.keys()))

# Dictionary(cluster_pts_q) = { cluster_name: [(x_q, y_q), ...] }
cluster_pts_q = {}
for gp, pt in zip(clusterized, list(ptQuery_ptTrain.keys())):
    if gp == -1: continue
    else:
        if gp not in cluster_pts_q:
            cluster_pts_q[gp] = [pt]
        else:
            cluster_pts_q[gp].append(pt)

cluster_pts_t = cp.deepcopy(cluster_pts_q)
for cluster in cluster_pts_t:
    for i, pt in enumerate(cluster_pts_t[cluster]):
        cluster_pts_t[cluster][i] = ptQuery_ptTrain[pt]

### Clusters' visualization

In [6]:
# for pts in cluster_pts_q.values():
#     for x, y in pts:
#         clust_img = cv.circle(query_img, (int(x), int(y)), radius=3, color=(0, 255, 0), thickness=-1)
#     imshow(clust_img)
#     clust_img = None

### Feature Homography

In [9]:
for cluster in cluster_pts_q:
    if cluster != 1:
        src = np.float32(cluster_pts_t[cluster]).reshape(-1, 1, 2)
        dst = np.float32(cluster_pts_q[cluster]).reshape(-1, 1, 2)

        M, _ = cv.findHomography(src, dst, cv.RANSAC, 5.)
        h, w = train_img_gray.shape
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        dst = [np.int32(dst)]

        res_img = cv.polylines(query_img, dst, True, (0, 0, 255), 3, cv.LINE_AA)
        res_img = cv.putText(res_img, 'Road Sign', tuple(dst[0][0][0]), cv.FONT_HERSHEY_DUPLEX, 1, (255, 0, 255), 1, cv.LINE_AA)

In [10]:
imshow(res_img)